# <div align="center" style="color: #ff5733;">Data Requested By Annastasia for Monthly Reporting</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
# Year and Month for which data is extracted
data_year = 2024
data_month = 10

In [3]:
# How many Applications applied to each month

sq = """
# loan applied
SELECT 
    DATE_TRUNC( startApplyDateTime,MONTH) as mm, 
    case when reloan_flag = 1  and loantype not like 'FLEXUP' then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
         
         else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as StartedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 

group by 1,2
order by 1 desc,2;
"""

StartedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in StartedApps are:\t {StartedApps.shape}")

Job ID 70da867d-607f-4ce2-9270-29070545fee0 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in StartedApps are:	 (213, 3)


In [4]:
# How many submitted applications to each month
sq = """
# loan Submitted
SELECT 
    DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime else termsAndConditionsSubmitDateTime end),MONTH) as mm,  
    case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as SubmittedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
group by 1,2
order by 1 desc,2;
"""

SubmittedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns of SubmittedApps are:\t {SubmittedApps.shape}")
SubmittedApps.sample(10)

Job ID cd00bdca-2cde-4f33-80f9-e31278b62554 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns of SubmittedApps are:	 (212, 3)


,mm,LoanProduct,SubmittedApps
142,2022-11-01,Flex,3866
204,NaT,ACL TSA,74
124,2023-03-01,SIL-Online(discontinued),1
54,2024-02-01,Flex-up,402
69,2023-12-01,SIL-Instore,16194
40,2024-04-01,ACL TSA,2
51,2024-03-01,SIL-Instore,9123
110,2023-05-01,Reloan,199
127,2023-02-01,Quick,14371
12,2024-09-01,Reloan,559


In [5]:
# Approved Loans for each month

sq = """
# Approved loans
SELECT 
DATE_TRUNC((case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then startApplyDateTime 
                  when reloan_flag = 1 and loantype not like 'FLEXUP' then startApplyDateTime else decision_date end),MONTH) as mm,
        case when reloan_flag = 1 and loantype not like 'FLEXUP'then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as ApprovedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
where 
(case when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then flagDisbursement 
      when reloan_flag = 1 and loantype not like 'FLEXUP' then flagDisbursement  else flagApproval end) = 1
group by 1,2
order by 1 desc,2
;
"""

ApprovedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in ApprovedApps are:\t {ApprovedApps.shape}")
ApprovedApps.head(10)

Job ID 03dd4d53-d61f-41a1-a3de-f9aa3f99ad02 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in ApprovedApps are:	 (174, 3)


,mm,LoanProduct,ApprovedApps
0,2024-11-01,Flex-up,21
1,2024-11-01,Quick,408
2,2024-11-01,Reloan,33
3,2024-11-01,SIL ZERO,171
4,2024-11-01,SIL-Instore,1662
5,2024-10-01,Flex-up,387
6,2024-10-01,Quick,4092
7,2024-10-01,Reloan,523
8,2024-10-01,SIL ZERO,1298
9,2024-10-01,SIL-Instore,12858


In [6]:
# Booked Loans for each month

sq = """
#Booked loans
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
            case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    count (distinct digitalLoanAccountId) as BookedApps
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
    where flagDisbursement = 1

group by 1,2
order by 1 desc,2
;
"""

BookedApps = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedApps are:\t {BookedApps.shape}")
BookedApps.head(10)

Job ID 51a3b320-25a7-4e9e-a8fd-6ef5b6e98e90 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedApps are:	 (173, 3)


,mm,LoanProduct,BookedApps
0,2024-11-01,Flex-up,37
1,2024-11-01,Quick,401
2,2024-11-01,Reloan,33
3,2024-11-01,SIL ZERO,164
4,2024-11-01,SIL-Instore,1630
5,2024-10-01,Flex-up,568
6,2024-10-01,Quick,3991
7,2024-10-01,Reloan,523
8,2024-10-01,SIL ZERO,1292
9,2024-10-01,SIL-Instore,12826


In [7]:
# Get the total booked amount for a month

sq = """
# Booked Amount
SELECT 
DATE_TRUNC( disbursementDateTime,MONTH) as mm, 
    case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    sum (disbursedLoanAmount) as BookedAmt
FROM 
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table` 
     where flagDisbursement = 1
group by 1,2
order by 1 desc,2
;
"""
BookedAmt = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in BookedAmt are:\t {BookedAmt.shape}")
BookedAmt.head(10)

Job ID 19f97b94-8b35-4095-be53-7626997b94a2 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in BookedAmt are:	 (173, 3)


,mm,LoanProduct,BookedAmt
0,2024-11-01,Flex-up,1.355715e+06
1,2024-11-01,Quick,4.769500e+06
2,2024-11-01,Reloan,5.941117e+05
3,2024-11-01,SIL ZERO,2.594587e+06
4,2024-11-01,SIL-Instore,1.560516e+07
5,2024-10-01,Flex-up,1.976254e+07
6,2024-10-01,Quick,4.987051e+07
7,2024-10-01,Reloan,1.187810e+07
8,2024-10-01,SIL ZERO,1.691628e+07
9,2024-10-01,SIL-Instore,1.243148e+08


In [8]:
BookedAmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173 entries, 0 to 172
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   mm           173 non-null    datetime64[us]
 1   LoanProduct  173 non-null    object        
 2   BookedAmt    173 non-null    float64       
dtypes: datetime64[us](1), float64(1), object(1)
memory usage: 4.2+ KB


In [9]:
raw_data = StartedApps.merge(SubmittedApps,how='outer').merge(ApprovedApps,how='outer').merge(BookedApps,how='outer').merge(BookedAmt,how='outer')

In [10]:
raw_data.tail(10)

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
205,2024-10-01,Flex-up,387,387,387,568,1.976254e+07
206,2024-10-01,Quick,62740,50582,4092,3991,4.987051e+07
207,2024-10-01,Reloan,583,550,523,523,1.187810e+07
208,2024-10-01,SIL ZERO,2574,2544,1298,1292,1.691628e+07
209,2024-10-01,SIL-Instore,21712,21436,12858,12826,1.243148e+08
210,2024-11-01,Flex-up,21,21,21,37,1.355715e+06
211,2024-11-01,Quick,5627,4675,408,401,4.769500e+06
212,2024-11-01,Reloan,42,37,33,33,5.941117e+05
213,2024-11-01,SIL ZERO,275,274,171,164,2.594587e+06
214,2024-11-01,SIL-Instore,2472,2448,1662,1630,1.560516e+07


In [11]:
print(f'{data_year}-{data_month:02d}-01')

2024-10-01


In [12]:
raw_data_current_month = raw_data[(raw_data['mm']>='2022-08-01') &(raw_data['mm']<=f'{data_year}-{data_month:02d}-01')]

In [13]:
raw_data_current_month

,mm,LoanProduct,StartedApps,SubmittedApps,ApprovedApps,BookedApps,BookedAmt
54,2022-08-01,Flex,5558,4732,594,599,1.198500e+07
55,2022-08-01,Quick,43764,30660,1910,1926,3.300100e+07
56,2022-08-01,SIL-Instore,967,956,456,457,1.137941e+07
57,2022-08-01,SIL-Online(discontinued),1192,652,41,41,1.197499e+06
58,2022-09-01,Big Loan,13,12,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...
205,2024-10-01,Flex-up,387,387,387,568,1.976254e+07
206,2024-10-01,Quick,62740,50582,4092,3991,4.987051e+07
207,2024-10-01,Reloan,583,550,523,523,1.187810e+07
208,2024-10-01,SIL ZERO,2574,2544,1298,1292,1.691628e+07


### Loan Account Tagging

In [14]:
sq="""
# Loan Tagging
SELECT
    loanAccountNumber,
    Case when reloan_flag = 1 and loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as LoanProduct,
    disbursementDateTime,
    (CASE WHEN new_loan_type = 'Flex-up' then
    LAG(new_loan_type) OVER (PARTITION BY customerId ORDER BY disbursementDateTime) END) AS OriginalLoanProduct
  FROM
    `prj-prod-dataplatform.risk_credit_mis.loan_master_table`
   where flagDisbursement=1
;
"""
loan_type = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in loan_type are: {loan_type.shape}")
loan_type.tail(10)

Job ID 5322ea24-9c12-493c-a281-4b448652d5d1 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in loan_type are: (227645, 4)


,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
227635,60829897530011,SIL-Instore,2024-11-01 12:04:47,None
227636,60829897660015,SIL-Instore,2024-11-01 15:40:48,None
227637,60829927770015,SIL-Instore,2024-11-02 14:16:33,None
227638,60829932040017,SIL-Instore,2024-11-02 15:55:43,None
227639,60829935210017,SIL-Instore,2024-11-02 17:13:57,None
227640,60829935870015,SIL-Instore,2024-11-02 17:33:26,None
227641,60829938540015,SIL-Instore,2024-11-02 19:24:23,None
227642,60829946260019,SIL-Instore,2024-11-03 09:18:45,None
227643,60829951820016,SIL-Instore,2024-11-03 12:18:21,None
227644,60829958770011,SIL-Instore,2024-11-03 15:35:14,None


In [15]:
loan_type[loan_type['loanAccountNumber']=='60828136490016']

,loanAccountNumber,LoanProduct,disbursementDateTime,OriginalLoanProduct
197672,60828136490016,SIL-Instore,2024-09-01 12:25:12,None


In [16]:
Loan_Acs_current_month = loan_type[(loan_type['disbursementDateTime'].dt.month == data_month) & (loan_type['disbursementDateTime'].dt.year == data_year)]

In [17]:
Loan_Acs_current_month = Loan_Acs_current_month[['loanAccountNumber', 'LoanProduct','OriginalLoanProduct']]

#### VAS

In [18]:
sq = """
# Vas 
SELECT 
    extract(year from disbursementDateTime) as year , 
    extract(month from disbursementDateTime) as month,
    Case when reloan_flag = 1 and b.loantype not like 'FLEXUP' and flagDisbursement = 1 then 'Reloan'
         when b.loantype = 'FLEXUP' and new_loan_type = 'Flex-up' and reloan_flag = 0 and flagDisbursement = 1 then 'Flex-up' 
                  else new_loan_type end as new_loan_type, 
    count(distinct  b.loanAccountNumber ) AS TotalSold,
   
   count(distinct CASE WHEN vas_flag = 'true' THEN b.loanAccountNumber END) AS SoldWithVAS
     FROM `prj-prod-dataplatform.dl_loans_db_raw.tdbk_digital_loan_application` a join
`prj-prod-dataplatform.risk_credit_mis.loan_master_table` b on a.loanAccountNumber = b.loanAccountNumber
where flagDisbursement = 1
group by 1,2 ,3 
order by 1 desc,2 desc,3
;
"""

vas_df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The rows and columns in vas_df are:\t {vas_df.shape}")
vas_df.head(10)

Job ID f4d8cd23-e0ce-41db-8d44-2df26df83b44 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The rows and columns in vas_df are:	 (173, 5)


,year,month,new_loan_type,TotalSold,SoldWithVAS
0,2024,11,Flex-up,37,12
1,2024,11,Quick,401,216
2,2024,11,Reloan,33,18
3,2024,11,SIL ZERO,164,89
4,2024,11,SIL-Instore,1630,1113
5,2024,10,Flex-up,568,277
6,2024,10,Quick,3991,2179
7,2024,10,Reloan,523,236
8,2024,10,SIL ZERO,1292,869
9,2024,10,SIL-Instore,12826,8663


In [19]:
vas_df_current_month = vas_df[(vas_df['year']==data_year)& (vas_df['month']==data_month)]

In [20]:
with pd.ExcelWriter(f'loan_accounts_tagging_{data_year}_{data_month:02d}_v1.xlsx') as writer:
    # Write each DataFrame to a specific sheet in the Excel file
    raw_data_current_month.to_excel(writer, sheet_name='raw_data', index=False)
    Loan_Acs_current_month.to_excel(writer, sheet_name='Loan Acs', index=False)
    vas_df_current_month.to_excel(writer, sheet_name='VAS', index=False)